In [43]:

import pandas as pd
import numpy as np
import os
import glob



# enter period and also enter manual entries in myml as required... e.g maintennace etc
userInput = '2019-10'
myml = (['2019-10-15','2019-10-15','2019-10-15'],
         ['oven 1','oven 2','oven 3'],
         [200,200,400])


# pick up all offset files
os.chdir("/Users/ronny/CBAStatements/Offset")
extension = 'csv'
offset_file = [i for i in glob.glob('*.{}'.format(extension))]
offset_csv = pd.concat([pd.read_csv(f, header=None) for f in offset_file ])
offset_csv['Source'] = 'Offset'

# pick up all card statements
extension = 'csv'
os.chdir("/Users/ronny/CBAStatements/Card")
card_file = [i for i in glob.glob('*.{}'.format(extension))]
card_csv = pd.concat([pd.read_csv(f, header=None) for f in card_file ])
card_csv['Source'] = 'Card'


df = pd.concat([offset_csv,card_csv],ignore_index=True)
df = df.drop([3],axis=1)
df.columns = ['Date','Amount','Description','Source']
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)




# replace some descriptions to other to help grouping and remove conflicts
df['Description'] = df['Description'].str.replace('Transfer to other Bank NetBank Admiral','Weekly - The Ponds ')
df['Description'] = df['Description'].str.replace('Bpay','B-Pay', case = False)
df.loc[df['Description'].str.contains('weekly pocket',case=False),'Description'] = 'Weekly Pocket Money'
df.loc[df['Description'].str.contains('pay cba card',case=False),'Description'] = 'Pay CBA Card'
df.loc[df['Description'].str.contains('deft rent',case=False),'Description'] = 'Deft Rent'
df.loc[df['Description'].str.contains('deft payments',case=False),'Description'] = 'Deft Payments'
df.loc[df['Description'].str.contains('bayside',case=False),'Description'] = 'Bayside Council'
df.loc[df['Description'].str.contains('sydney water',case=False),'Description'] = 'Sydney Water'
#df.loc[df['Description'].str.contains('Transfer to xx0765',case=False),'Description'] = 'Pay CBA Card'




In [44]:


# Put default value first for card before doing grouping.... left over needs to be checked later
df.loc[df['Source']=='Card','Group'] = 'Z.Card - Other'


# grouping parameters... please add to mystr and mygroup accordingly
mystr = (['Salary'],
         ['harcourts','hills prestige','hill'],
         ['repay 778047934','repay 876891821','weekly - the ponds'],
         ['weekly pocket'],
         ['deft payments'],
         ['deft rent'],
         ['slick','carena'],
         ['repay 778047926'],
         ['bayside'],
         ['sydney water'],
         ['westpac life'],
         ['sandra low','krishnan','pharmacy','ultrasound','oral care','sheng ruan','chemist','terry white','pline'],
         ['general ins'],
         ['gio','nrma','aami','service nsw'],
         ['medibank','bupa'],
         ['bp','caltex'],
         ['optus'],
         ['AGL'],
         ['kogan','bing lee','jb','harvey','uniqlo','h&m','kmart','bunnings','target','decathlon',
          'reject shop','baby bunting','hair'],
         ['opal'],
         ['netflix'],
         ['tpg'],
         ['coles','woolworths','randwick oriental','eastgate','harris farm','aldi','tong li'],
         ['top one','super cheap'],
         ['ymca'],
         ['helping hand'],
         ['srg'],
         ['woollahra ps'],
         ['medan ciak', 'ayam','churchill','golden','malatang','shalom','guzman','pork','tonkatsu','chinese','grilld'
         ,'star bar','master wong','my delight','mumbai','mcdonald','tea','hot','japan','hoyts','groupon'
         ,'compass wbc','28 degrees','tonkotsuya','lucky charm','mappen','myeong'],
         ['interesting','virgin','nutrition'],
         ['transfer from'],
         ['transfer to'],
         ['payment received'],
         ['b-pay'],
        )


mygrp = (['A. Salary'],
         ['B. Property - The Ponds Income'],
         ['B. Property - The Ponds Int'],
         ['B. Pocket Money'],
         ['B. Property - Wolli Strata'],
         ['B. Property - Bondi Rent'],
         ['B. Property - Wolli Income'],
         ['B. Property - Wolli Int'],
         ['B. Property - Wolli Council'],
         ['B. Property - Wolli Water'],
         ['C. Insurance - Life'],
         ['C1. Health'],
         ['C. Insurance - Properties'],
         ['C. Insurance - car'],
         ['C. Insurance - Health'],
         ['D. Petrol'],
         ['D. Mobile & Fetch'],
         ['D. Gas & Electricity'],
         ['D. Shopping'],
         ['D. Opal Card'],
         ['D. Netflix'],
         ['D. Internet'],
         ['D. Groceries'],
         ['E. Car - Service'],
         ['Phoebe - Swimming'],
         ['Phoebe - After Care'],
         ['Phoebe - Muaythai'],
         ['Phoebe - School'],
         ['Rest & Entertainment'],
         ['Ronny - Sport'],
         ['Z. Transfer From'],
         ['Z. Transfer To'],
         ['Z. Card Payment'],
         ['Z. Bpay Payment'])


# apply the grouping
for k in range(len(mygrp)):
    count = len(mystr[k])
    for s in range(count):
        df.loc[df['Description'].str.contains(mystr[k][s],case=False),'Group'] = mygrp[k]
        

# create period using string format 'YYYY-MM'        
df['Period'] = df['Date'].dt.strftime("%Y-%m")

# filter the last 6 months data only
#df1 = df1[(df1.Date > '2019-09-30') & (df1.Date < '2020-11-01')] 


# create report
actual = pd.pivot_table(df, values='Amount', index=['Group'],
                    columns=['Period'], aggfunc=np.sum)



In [45]:

# update the list with your forcast numbers
# format is fr, to, freqDate, offsetDate, amt, description

myval = (['2019-10-10','2021-01-31','14D',0,3156,'A. Salary'],
         ['2019-10-01','2019-10-31','14D',0,2103,'A. Salary'],
         ['2019-09-27','2021-01-31','7D',0,-300,'B. Pocket Money'],
         ['2019-09-19','2021-01-31','14D',0,-1300,'B. Property - Bondi Rent'],
         ['2019-08-26','2021-01-31','7D',0,650,'B. Property - The Ponds Income'],
         ['2019-09-25','2021-01-31','7D',0,-889.85,'B. Property - The Ponds Int'],
         ['2019-10-01','2021-01-31','QS-Aug',0,-327.29,'B. Property - Wolli Council'],
         ['2019-09-24','2021-01-31','7D',0,520,'B. Property - Wolli Income'],
         ['2019-09-01','2019-10-08','MS',7,-1145.19,'B. Property - Wolli Int'],
         ['2019-10-09','2021-01-31','7D',0,-311,'B. Property - Wolli Int'],
         ['2019-09-01','2021-01-31','QS-Sep',0,-1200,'B. Property - Wolli Strata'],
         ['2019-10-01','2021-01-31','QS-Oct',0,-173.25,'B. Property - Wolli Water'],
         ['2019-10-01','2020-02-28','MS',0,-421.45,'C. Insurance - Health'],
         ['2020-02-28','2021-02-28','MS',0,-150.45,'C. Insurance - Health'],
         ['2019-09-01','2021-01-31','AS-Sep',0,-870,'C. Insurance - Life'],
         ['2019-09-01','2021-01-31','AS-Sep',0,-1850,'C. Insurance - car'],
         ['2019-10-01','2021-01-31','MS',0,-140.26,'C. Insurance - Properties'],
         ['2019-10-01','2020-01-31','MS',0,-600,'C1. Health'],
         ['2019-10-01','2021-01-31','QS-Sep',0,-750,'D. Gas & Electricity'],
         ['2019-10-01','2021-01-31','MS',0,-350,'D. Groceries'],
         ['2019-10-01','2021-01-31','MS',0,-59.99,'D. Internet'],
         ['2019-10-01','2021-01-31','MS',0,-60,'D. Mobile & Fetch'],    
         ['2019-10-01','2021-01-31','MS',0,-17.99,'D. Netflix'], 
         ['2019-10-01','2021-01-31','MS',0,-20,'D. Opal Card'], 
         ['2019-10-01','2021-01-31','MS',0,-75,'D. Petrol'],
         ['2019-10-01','2021-01-31','MS',0,-50,'D. Shopping'],
         ['2019-10-01','2021-01-31','QS-Oct',0,-300,'E. Car - Service'],
         ['2019-09-26','2021-01-31','14D',0,-81.86,'Phoebe - After Care'],
         ['2019-09-26','2021-01-31','QS-Feb',0,-200,'Phoebe - Muay Thai'],
         ['2019-09-26','2021-01-31','QS-Oct',0,-70,'Phoebe - School'],
         ['2019-10-01','2021-01-31','MS',0,-80,'Phoebe - Swimming'],
         ['2019-10-01','2021-01-31','MS',0,-350,'Rest & Entertainment'],
         ['2019-10-09','2021-01-31','14D',0,-47,'Ronny - Sport'],
         ['2018-10-09','2021-01-31','AS-Oct',0,8000,'Tax Return'])
         
         
         
         
# loop all the description and create dataframe 
for i in range(len(myval)):
        fr = myval[i][0]
        to = myval[i][1]
        frq = myval[i][2]
        offset = myval[i][3]
        amt = myval[i][4]
        desc = myval[i][5]
        
        mydate=pd.date_range(fr,to,freq=frq) + pd.DateOffset(days=offset)
        myamt = np.repeat(amt,len(mydate))
        mydesc = desc
        
        if i ==0 : # create the main dataframe the first time
            df1 = pd.DataFrame ({'Date':mydate, 'Desc':mydesc,'Amount':myamt})
        else:      # create tmp and append to main dataframe
            tmp = pd.DataFrame ({'Date':mydate, 'Desc':mydesc,'Amount':myamt})
            df1 = df1.append(tmp)
            
            
            
# insert user manual entries from the input to dataframe

tmp = pd.DataFrame({'Date':myml[0],'Desc':myml[1],'Amount':myml[2]})
tmp['Date']= pd.to_datetime(tmp['Date'])
df1 = df1.append(tmp)         

            
df1['Period'] = df1['Date'].dt.strftime("%Y-%m")

# filter the last 6 months data only

#df1 = df1[(df1.Date > '2019-09-30') & (df1.Date < '2019-10-31')] 


# produce details report
forecast = pd.pivot_table(df1, values='Amount', index=['Desc'],
                    columns=['Period'], aggfunc=np.sum)



In [46]:

a = actual[userInput]
b = forecast[userInput]

remove = ['Z. Transfer To','Z.Card - Other','Z. Bpay Payment','Z. Card Payment','Z. Transfer From']
rpt_actual = a.drop(remove)


remove = ['Tax Return']
rpt_target = b.drop(remove)

In [47]:
pd.DataFrame({'actual':rpt_actual,'target':rpt_target,'var':rpt_actual-rpt_target})

,actual,target,var
A. Salary,7364.64,12621.00,-5256.36
B. Pocket Money,-600.00,-1200.00,600.00
B. Property - Bondi Rent,-1300.00,-3900.00,2600.00
B. Property - The Ponds Income,611.50,2600.00,-1988.50
B. Property - The Ponds Int,-2263.74,-4449.25,2185.51
B. Property - Wolli Council,NaN,NaN,NaN
B. Property - Wolli Income,833.40,2600.00,-1766.60
B. Property - Wolli Int,-1419.25,-2389.19,969.94
B. Property - Wolli Strata,NaN,NaN,NaN
B. Property - Wolli Water,NaN,-173.25,NaN


In [48]:
actual.loc[:,'2019-07':]

Period,2019-09,2019-10
Group,,
A. Salary,10521.44,7364.64
B. Pocket Money,-1200.00,-600.00
B. Property - Bondi Rent,-2600.00,-1300.00
B. Property - The Ponds Income,3259.05,611.50
B. Property - The Ponds Int,-4053.37,-2263.74
B. Property - Wolli Income,562.85,833.40
B. Property - Wolli Int,-1145.19,-1419.25
B. Property - Wolli Strata,-1200.00,NaN
C. Insurance - Health,-421.45,NaN


In [49]:
forecast.loc[:,'2019-10':'2020-03']

Period,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03
Desc,,,,,,
A. Salary,12621.00,6312.00,6312.00,9468.00,6312.00,6312.00
B. Pocket Money,-1200.00,-1500.00,-1200.00,-1500.00,-1200.00,-1200.00
B. Property - Bondi Rent,-3900.00,-2600.00,-2600.00,-2600.00,-2600.00,-2600.00
B. Property - The Ponds Income,2600.00,2600.00,3250.00,2600.00,2600.00,3250.00
B. Property - The Ponds Int,-4449.25,-3559.40,-3559.40,-4449.25,-3559.40,-3559.40
B. Property - Wolli Council,NaN,-327.29,NaN,NaN,-327.29,NaN
B. Property - Wolli Income,2600.00,2080.00,2600.00,2080.00,2080.00,2600.00
B. Property - Wolli Int,-2389.19,-1244.00,-1244.00,-1555.00,-1244.00,-1244.00
B. Property - Wolli Strata,NaN,NaN,-1200.00,NaN,NaN,-1200.00


In [50]:
remove = ['Z. Transfer To','Z.Card - Other','Z. Bpay Payment','Z. Card Payment','Z. Transfer From']
actual.loc[remove,:]

Period,2019-09,2019-10
Group,,
Z. Transfer To,-7280.0,-4355.00
Z.Card - Other,NaN,-38.27
Z. Bpay Payment,-27.0,-299.34
Z. Card Payment,5580.0,4855.00
Z. Transfer From,4054.0,2614.00


In [51]:

# check the details on those weird categories
df[(df['Group'].isin(['Z.Card - Other','Z. Bpay Payment'])) & (df['Period'].isin([userInput]))]

,Date,Amount,Description,Source,Group,Period
3,2019-10-15,-299.34,COLES MASTERCARD CommBank app B-Pay 8862 5...,Offset,Z. Bpay Payment,2019-10
72,2019-10-07,-11.02,COLIN'S ZETLAND PTY ZETLAND NSW,Card,Z.Card - Other,2019-10
77,2019-10-05,-16.00,SPOTLIGHT 028 BONDI JUNCTINSW,Card,Z.Card - Other,2019-10
79,2019-10-05,-11.25,Tender Gourmet Butche Bondi JunctiNSW,Card,Z.Card - Other,2019-10


In [52]:
# query specific description against actual data
df[df['Description'].str.contains('gio',case=False)]

,Date,Amount,Description,Source,Group,Period
112,2019-09-18,-437.16,GIO CTP BRISBANE QLD,Card,C. Insurance - car,2019-09


In [53]:
# check property income
df[(df['Group'].isin(['B. Property - Wolli Income','B. Property - The Ponds Income'])) & (df['Period'].isin([userInput]))]

,Date,Amount,Description,Source,Group,Period
1,2019-10-15,611.5,Direct Credit 536638 M T Flanagan P L Hills Pr...,Offset,B. Property - The Ponds Income,2019-10
2,2019-10-15,833.4,Direct Credit 010842 SLICKPROPEMAP Mid Mnth Rn...,Offset,B. Property - Wolli Income,2019-10
